# Prophet - Predecir el valor de Bitcoin


Prophet es una de las librerías más avanzadas para predecir series temporales desarrollada por Facebook. Te enseñaremos a como entrenar un modelo con Prophet, a optimizarlo y a utilizarlo para realizar predicciones futuras. En este ejercicio vamos a practicar a predecir el valor de Bitcoin, una criptomoneda. Es la criptomoneda que le ha marcado el camino a todas las demás que llegaron después utilizando su tecnología.

<hr/>
<div class="alert alert-success alertsuccess" style="margin-top: 20px">
[Tip]: Para ejecutar el código de Python en la celda de código a continuación, haz clic en la celda para seleccionarla y presiona <kbd>Shift</kbd> + <kbd>Enter</kbd>.
</div>
<hr/>


In [34]:
# Instalamos el modelo predictivo Prophet
!python -m pip install prophet --quiet
# De aquí vamos a descargar los datos
!pip install yfinance --quiet

In [33]:
#Importamos las dependencias
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
import warnings

warnings.filterwarnings('ignore')

pd.options.display.float_format = '${:,.2f}'.format

<h3 id="version">Cargar histórico de datos</h3>


<p>
    Para descargarnos la serie temporal de BTC lo vamos a hacer desde Yahoo Finance. En concreto vamos a descargar el histórico desde 2016 hasta la actualidad.
</p>



In [31]:
# Fecha de inicio es un string 2016-01-01 y fecha de fin (hoy) en formato texto '%Y-%m-%d'
today = '2023-10-04'
start_date = '2016-01-01'
# Descargar el dataframe
btc_df = yf.download('BTC-USD',start_date, today).reset_index()

btc_df.tail()

[*********************100%%**********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
2828,2023-09-29,"$27,024.84","$27,225.94","$26,721.76","$26,911.72","$26,911.72",10396435377
2829,2023-09-30,"$26,911.69","$27,091.79","$26,888.97","$26,967.92","$26,967.92",5331172801
2830,2023-10-01,"$26,967.40","$28,047.24","$26,965.09","$27,983.75","$27,983.75",9503917434
2831,2023-10-02,"$27,976.80","$28,494.46","$27,347.79","$27,530.79","$27,530.79",19793041322
2832,2023-10-03,"$27,508.25","$27,667.19","$27,216.00","$27,429.98","$27,429.98",11407814187


In [9]:
# TAREA: Validamos que no hay datos vacíos
btc_df.isnull().sum()

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [39]:
#filtramos por 'Date' y 'Open'
df = btc_df[['Date', 'Open']].rename(columns={'Date': 'ds', 'Open': 'y'})

In [40]:
# Crear la gráfica del precio de apertura

x = df["ds"]
y = df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Le ponemos el título
fig.update_layout(
    title_text="Serie temporal de Bitcoin Precio de Apertura",
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1a", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

<h3 id="version">Entrenar y predecir el modelo</h3>


<p>
    Inicializamos el Prophet y lo entramos pasandole el dataframe
</p>



Documentacion oficial de Prophet [GitHub](https://facebook.github.io/prophet/docs/quick_start.html)

In [41]:
#TAREA: Inicializa Prophet en la variable m con seasonality_mode en modo "multiplicative"
m = Prophet()

In [ ]:
#TAREA: Entrena el modelo m con el dataframe de datos df
m.fit(df)

<p>
    Le decimos al modelo m los días en futuro que queremos predecir, en este caso el próximo año.
</p>



In [ ]:
# TAREA: generar un dataframe con los días que tenemos y los 365 días siguientes a hoy para predecirlo
# PISTA: puedes usar la función del prophet make_future_dataframe()
future =
future.tail()

In [45]:
future = m.make_future_dataframe(periods=365)
future.tail()

,ds
3193,2024-09-28
3194,2024-09-29
3195,2024-09-30
3196,2024-10-01
3197,2024-10-02


In [ ]:
# En el caso que hayas elegido un valor o fondo tradicional, es decir, que no opere los fines
# de semana, descomenta la siguiente línea:

# future = future[ future['ds'].dt.dayofweek < 5 ] # Nos elimina los fines de semana de 'ds' para no predecirlos
future

,ds
0,2016-01-01
3,2016-01-04
4,2016-01-05
5,2016-01-06
6,2016-01-07
...,...
2859,2023-10-30
2860,2023-10-31
2861,2023-11-01
2862,2023-11-02


<p>
    Al modelo entrenado le pasamos el dataframe a predecir.
</p>



In [47]:
#Utiliza el modelo m para predecir el dataframe future
forecast = m.predict(future)

In [48]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
3193,2024-09-28,"$12,423.95","$-6,388.90","$28,229.35"
3194,2024-09-29,"$12,392.50","$-6,438.28","$26,699.33"
3195,2024-09-30,"$12,350.12","$-5,402.42","$28,307.63"
3196,2024-10-01,"$12,324.29","$-6,315.61","$28,064.05"
3197,2024-10-02,"$12,285.04","$-5,568.68","$28,006.37"


In [49]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')

forecast[forecast['ds'] == next_day]['yhat'].item()

19401.558375034572

<p>
    Gráfica con el modelo entrenado y los valores de un año en futuro predecidos.
</p>



In [50]:
plot_plotly(m, forecast)

In [ ]:
plot_components_plotly(m, forecast)